# Importing all the necessary modules

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import numpy as np


# Downloading the CIFAR10 dataset

## The CIFAR-10 dataset

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.




In [ ]:
def download_CIFAR10_dataset():
    """Downloads the CIFAR-10 dataset in the data folder and returns training data and testing data."""
    train_data = datasets.CIFAR10(
        root = "data",
        download=True,
        train=True,
        transform=ToTensor()
    )
    validation_data = datasets.CIFAR10(
        root = "data",
        download=True,
        train=False,
        transform=ToTensor()
    ) 

    return train_data,validation_data

In [ ]:
dow